In [26]:
%reload_ext autoreload
%autoreload 2

In [27]:
import pandas as pd
from pathlib import Path

from mbp.webscraping import activate_web_driver, get_team_page
from mbp.data import download_team_data, get_team_games_for_year
from mbp.paths import SEASONS_DIR, RAW_DATA_DIR
from mbp.models import TeamYear, TeamGame

In [28]:
team_name = "Arizona"
year = 2022

In [29]:
driver = activate_web_driver("firefox")

# Read team names
df = pd.read_csv(f"{RAW_DATA_DIR}/mbb_team_names_to_number.csv", index_col=0)

In [30]:
games_df = get_team_games_for_year(driver, df, team_name, year)
games_df.head()

/Users/auser/work/aiml/mine/ncaa-mb-prediction/mbp/mbp/webscraping.py:236: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '95' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  games_df2.at[idx, "team_score"] = score_parts[0]
/Users/auser/work/aiml/mine/ncaa-mb-prediction/mbp/mbp/webscraping.py:237: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '78' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  games_df2.at[idx, "opp_score"] = score_parts[1]


,opponent,result,attendance,datetime,win,team_score,opp_score,home
0,Southern U.,95-78,"13,485",2022-11-11,1.0,95,78,1.0
1,Utah Tech,104-77,"12,752",2022-11-17,1.0,104,77,1.0
2,Cincinnati,101-93,"2,400",2022-11-21,1.0,101,93,0.0
3,San Diego St.,87-70,"2,400",2022-11-22,1.0,87,70,0.0
4,Creighton,81-79,"2,400",2022-11-23,1.0,81,79,0.0


In [31]:
# Download Arizona data
download_team_data(team_name, year, True)

/Users/auser/work/aiml/mine/ncaa-mb-prediction/mbp/mbp/webscraping.py:236: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '95' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  games_df2.at[idx, "team_score"] = score_parts[0]
/Users/auser/work/aiml/mine/ncaa-mb-prediction/mbp/mbp/webscraping.py:237: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '78' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  games_df2.at[idx, "opp_score"] = score_parts[1]


In [32]:
# Get their games for the year
year_dir = Path(SEASONS_DIR / str(year))
team_dir = year_dir / team_name
games_df = pd.read_csv(team_dir / "games.csv")

for opp in games_df['opponent']:
    opponent_games = year_dir / opp / "games.csv"
    if not opponent_games.exists():
        print(opp)
        download_team_data(opp, year)

In [33]:
from mbp.models import TeamYear, TeamGame

In [34]:
az = TeamYear("Arizona", 2022)
az.download_updated_data(False)

games = az.get_games()

In [12]:
from datetime import datetime, timedelta, date
today = date.today()
today = today + timedelta(days=-290)
print(today.strftime("%Y-%m-%d"))

df2 = games.copy()
df2['datetime'] = pd.to_datetime(df2['datetime'])
df2['difference'] = (df2['datetime'].dt.date - today)
# next_row = df.loc[df['difference'] == df['difference'].min()]
# print(df2.loc[df2['TimeDifference'].min()])
future_dates = df2[df2['difference'] > timedelta(days=0)]
next_future_date = future_dates.loc[future_dates['difference'].idxmin()]
next_future_date

2023-01-10


opponent               Oregon St.
result                      86-74
attendance                  4,126
datetime      2023-01-12 00:00:00
win                           1.0
team_score                     86
opp_score                      74
home                          0.0
difference        2 days, 0:00:00
Name: 15, dtype: object

In [114]:
from mbp.models import get_next_game

today = date.today()
last_year_date = today + timedelta(days=-170)

az.get_next_opponent_or_last(last_year_date)
game = get_next_game(az, year, last_year_date)
game_stats = game.get_game_stats()
# # game.get_players()
game_stats

,player,pos,g,mp,fgm,fga,3fg,3fga,ft,fta,...,drebs,tot reb,ast,to,stl,blk,fouls,dq,tech fouls,team
0,"Pierce, Caden",F,1,33:49,2,6,0,4,2,3,...,6,8,1,2,1,1,4,0,0,Princeton
1,"Kellman, Keeshawn",F,1,13:41,4,4,0,0,0,0,...,3,4,0,1,0,2,3,0,0,Princeton
2,"Allocco, Matt",G,1,38:07,3,12,0,3,0,0,...,2,3,2,1,2,0,0,0,0,Princeton
3,"Langborg, Ryan",G,1,31:06,4,13,0,6,0,0,...,1,2,1,2,1,2,2,0,0,Princeton
4,"Evbuomwan, Tosan",F,1,36:23,7,15,0,1,1,2,...,7,7,4,2,1,1,3,0,0,Princeton
5,"Peters, Blake",G,1,15:16,3,6,3,5,0,0,...,3,3,0,1,1,0,0,0,0,Princeton
6,"Lee, Xaivian",G,1,2:14,0,1,0,1,0,0,...,1,1,0,1,0,0,1,0,0,Princeton
7,"Martini, Zach",F,1,29:24,3,7,1,5,0,0,...,5,7,1,1,2,0,2,0,0,Princeton
8,"Tubelis, Azuolas",F,1,37:18,9,20,1,2,3,4,...,4,5,1,6,4,1,4,0,0,Arizona
9,"Henderson Jr., Cedric",F,1,27:01,1,4,0,2,0,0,...,4,4,1,0,1,0,3,0,0,Arizona


In [115]:
princeton_players = game_stats[game_stats['team'] == 'Princeton']
all_princeton_players = [p for p in game.team_b.get_roster()['player']]
# Players who played in the game with minutes
active_princeton_players = [x for x in [p for p in princeton_players['player'] if p in all_princeton_players] if x != 'Team']
arizona_players = game_stats[game_stats['team'] == 'Arizona']
all_arizona_players = [p for p in game.team_a.get_roster()['player']]
active_arizona_players = [x for x in [p for p in arizona_players['player'] if p in all_arizona_players] if x != 'Team']
active_princeton_players, active_arizona_players


(['Pierce, Caden',
  'Kellman, Keeshawn',
  'Allocco, Matt',
  'Langborg, Ryan',
  'Evbuomwan, Tosan',
  'Peters, Blake',
  'Lee, Xaivian',
  'Martini, Zach'],
 ['Tubelis, Azuolas',
  'Henderson Jr., Cedric',
  'Ramey, Courtney',
  'Kriisa, Kerr',
  'Ballo, Oumar',
  'Veesaar, Henri',
  'Boswell, Kylan',
  'Larsson, Pelle'])

In [184]:
stats = game.get_game_stats()
stats.head()

,player,pos,g,mp,fgm,fga,3fg,3fga,ft,fta,...,drebs,tot reb,ast,to,stl,blk,fouls,dq,tech fouls,team
0,"Pierce, Caden",F,1,33:49,2,6,0,4,2,3,...,6,8,1,2,1,1,4,0,0,Princeton
1,"Kellman, Keeshawn",F,1,13:41,4,4,0,0,0,0,...,3,4,0,1,0,2,3,0,0,Princeton
2,"Allocco, Matt",G,1,38:07,3,12,0,3,0,0,...,2,3,2,1,2,0,0,0,0,Princeton
3,"Langborg, Ryan",G,1,31:06,4,13,0,6,0,0,...,1,2,1,2,1,2,2,0,0,Princeton
4,"Evbuomwan, Tosan",F,1,36:23,7,15,0,1,1,2,...,7,7,4,2,1,1,3,0,0,Princeton


In [195]:
game.team_b.get_games(True)

/Users/auser/work/aiml/mine/ncaa-mb-prediction/mbp/mbp/webscraping.py:236: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '73' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  games_df2.at[idx, "team_score"] = score_parts[0]
/Users/auser/work/aiml/mine/ncaa-mb-prediction/mbp/mbp/webscraping.py:237: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '74' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  games_df2.at[idx, "opp_score"] = score_parts[1]


Navy
2022-23 MBB Veterans Classic has a newline
['Navy', '2022-23 MBB Veterans Classic']
Penn
2023 has a newline
['Penn', '2023']
Yale
2023 has a newline
['Yale', '2023']


,opponent,result,attendance,datetime,win,team_score,opp_score,home
0,Navy,73-74,"2,889",2022-11-11,0.0,73,74,0.0
1,UMBC,94-64,"2,097",2022-11-14,1.0,94,64,0.0
2,Marist,62-55,823,2022-11-19,1.0,62,55,0.0
3,Army West Point,74-66,"1,551",2022-11-24,1.0,74,66,0.0
4,Northeastern,56-54,"2,074",2022-11-26,1.0,56,54,0.0
5,Cairn,92-58,742,2022-11-30,1.0,92,58,1.0
6,Drexel,83-63,"1,412",2022-12-03,1.0,83,63,0.0
7,Lafayette,69-58,920,2022-12-06,1.0,69,58,1.0
8,Monmouth,91-54,"1,372",2022-12-10,1.0,91,54,1.0
9,Iona,64-70,496,2022-12-13,0.0,64,70,0.0


In [116]:
team_a = game_stats.loc[game_stats['team'] == game.team_a.team_name]
all_team_a_players = game.team_a.get_roster()
player_names = [p for p in all_team_a_players['player']]
active_players = [p for p in team_a['player'] if p in player_names]
active_players

['Tubelis, Azuolas',
 'Henderson Jr., Cedric',
 'Ramey, Courtney',
 'Kriisa, Kerr',
 'Ballo, Oumar',
 'Veesaar, Henri',
 'Boswell, Kylan',
 'Larsson, Pelle']

In [120]:
stats = game.team_a.get_stats()
print(f"colums: {stats.columns}")
df1 = game.team_a.get_roster()
roster_stats = game.team_a.get_roster_stats()
roster_stats

roster_stats['3fg']

colums: Index(['jersey', 'player', 'yr', 'pos', 'ht', 'gp', 'gs', 'mp', 'g', 'fgm',
       'fga', 'fg%', '3fg', '3fga', '3fg%', 'ft', 'fta', 'ft%', 'pts', 'avg',
       'orebs', 'drebs', 'tot reb', 'avg.1', 'ast', 'to', 'stl', 'blk',
       'fouls', 'dbl dbl', 'trpl dbl', 'dq', 'tech fouls', 'effective fg pct.',
       'year'],
      dtype='object')


0      0
1      1
2     12
3      0
4     30
5     37
6     83
7      0
8      3
9     31
10     0
11     0
12    83
13    10
14     0
15     3
16     0
Name: 3fg, dtype: int64

In [123]:
team_b

6.25